# HW2 - Q4: Training a robust model & Tighter Certification (10 points)

**Keywords**: Adversarial Robustness Training

**About the dataset**: \
The [MNIST](https://en.wikipedia.org/wiki/MNIST_database) database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems.\
The MNIST database contains 70,000 labeled images. Each datapoint is a $28\times 28$ pixels grayscale image.

**Agenda**:
* In this programming challenge, you will train a 2-hidden layer neural network which is robust to adversarial attacks. 
* You will train models on adversarial examples generated using both FGSM and PGD.
* Finally, you will compare the robustness of a standard (non-robust) model vs. the robust models using both IBP and FastLin bound Algorithms.


**Note:**
* **It is recommended that you use GPU hardware accelaration for this question.**
* A note on working with GPU:
  * Take care that whenever declaring new tensors, set `device=device` in parameters. 
  * You can also move a declared torch tensor/model to device using `.to(device)`. 
  * To move a torch model/tensor to cpu, use `.to('cpu')`
  * Keep in mind that all the tensors/model involved in a computation have to be on the same device (CPU/GPU).
* Run all the cells in order.
* **Do not edit** the cells marked with !!DO NOT EDIT!!
* Only **add your code** to cells marked with !!!! YOUR CODE HERE !!!!
* Do not change variable names, and use the names which are suggested.



---



---



## Preprocessing:

In [ ]:
# install this library
!pip install gdown

In [ ]:
# !!DO NOT EDIT!!
# imports 
import torch
import torch.nn as nn
import numpy as np
import requests
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import gdown
from zipfile import ZipFile

# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# loading the dataset full MNIST dataset
mnist_train = datasets.MNIST("./data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("./data", train=False, download=True, transform=transforms.ToTensor())

mnist_train.data = mnist_train.data.to(device)
mnist_test.data = mnist_test.data.to(device)

mnist_train.targets = mnist_train.targets.to(device)
mnist_test.targets = mnist_test.targets.to(device)

# reshape and min-max scale
X_train =  (mnist_train.data.reshape((mnist_train.data.shape[0], -1))/255).to(device)
y_train = mnist_train.targets
X_test = (mnist_test.data.reshape((mnist_test.data.shape[0], -1))/255).to(device)
y_test = mnist_test.targets

# load pretrained and dummy model
url_nn_model = 'https://bit.ly/3sKvyOs'
url_models   = 'https://bit.ly/3lsVcDn'
gdown.download(url_nn_model, 'nn_model.pt')
gdown.download(url_models, 'models.zip')
ZipFile("models.zip").extractall("./")

from model import NN_Model
from test_model import Test_Model
nn_model = torch.load("./nn_model.pt").to(device)
print('Pretrained model (nn_model):', nn_model)

test_model = Test_Model()
print('Dummy model (test_model):', test_model)

# This will save the linear layers of the neural network model in a ordered list
# Eg:
# to access weight of first layer: model_layers[0].weight
# to access bias of first layer: model_layers[0].bias
model_layers = [layer for layer in nn_model.children()] # for nn_model
test_model_layers = [layer for layer in test_model.children()] # for dummy model

# first few examples
example_data = mnist_test.data[:18]/255
example_data_flattened  = example_data.view((example_data.shape[0], -1)).to(device) # needed for training
example_labels = mnist_test.targets[:18].to(device)



---



---




### **Adversarial training:** In order to train robust models, the most intuitive strategy is to train on adversarial examples.
* The adversarial (robust) loss function is defined as:
$\underset{\theta}{minimize} \frac{1}{|S|}\sum_{x,y\in S} \underset{∥δ∥≤ϵ}{max}\,ℓ(h_θ(x+δ),y)$, \
where $\theta$ are the learnable parameters, $S$ is the set of training examples with $x$ representing the input example and $y$ the ground truth label, $h_\theta$ is the hypothesis (neural network model), $\delta$ is the attack perturbation, and $\epsilon$ is the attack budget.

* This is also known as the min-max loss function. The gradient descent step now becomes:\
$\theta:=\theta-\frac{\alpha}{|B|}\sum_{x,y\in B} ∇_θ \underset{∥δ∥≤ϵ}{max}\,ℓ(h_θ(x+δ),y)$,\
where $B$ is the mini-batch and $\alpha$ is the learning rate. 

* Now the question becomes how to find the inner term: $∇_θ \underset{∥δ∥≤ϵ}{max}\,ℓ(h_θ(x+δ),y)$ of the gradient descent step. For this, we can use **Danskin’s Theorem**, which states that to compute the (sub)gradient of a function containing a max term, we need to simply 1) find the maximum, and 2) compute the normal gradient evaluated at this point. It holds only when you have the exact maximum.  Note that it is not possible to solve the inner maximization problem exactly (NP-hard). However, the better job we do of solving the inner maximization problem, the closer it seems that Danskin’s theorem starts to hold. That is why we can re-use methods such as FGSM/PGD to find approximate worst case examples. In other words, we can perform the attack to find $δ^{*} = \underset{∥δ∥≤ϵ}{argmax}ℓ(h_θ(x+δ),y)$, and then compute this term at the perturbed image: $∇_θℓ(h_θ(x+δ^{*}),y)$.

In summary, we would create an adversarial example for each datapoint in the mini-batch and add the loss corresponding to that example to the gradient.  



---



### **(a) Setup:** (2 points)
### **#1.**Get the attack functions `fgsm` and `pgd` that you defined in the HW2-Q3 (a) and (b). Modify the `pgd` function to start with random values of `delta` instead of zeros.

In [ ]:
#######
# !!! YOUR CODE HERE !!!

#######

### **#2.** Create a 2-hidden-layer neural network model in pytorch. The input should be the size of the flattened MNIST image, and output layer should be of size 10, which is the number of target labels. Each of the two hidden layers should be of size 1024 with ReLU activations between each subsequent layer except the last layer.

You can refer to the structure of `nn_model` from HW2-Q3. It should be similar to that.

In [ ]:
#######
# !!!! YOUR CODE HERE !!!!

#######

### **#3.** Define a function `train_torch_model_adversarial` that takes as input an initialized torch model (`model`), batch size (`batch_size`), initialized loss (`criterion`), max number of epochs (`max_epochs`), training data (`X_train, y_train`), learning rate (`lr`), tolerance for stopping (`tolerance`), adversarial strategy (`adversarial_strategy`: `None/'fgsm'/'pgd'`), and attack budget(`epsilon`). 

### **Note**: 
* If `adversarial_strategy` is `None`, don't train on adversarial examples.
* This function will return a tuple of `(model, losses)`, where `model` is the trained model, and `losses` are a list of tuple of loss logged every epoch. 
* The only difference from the function `train_torch_model` that you wrote in HW2-Q2 (b) is that you also add gradient of adversarial example. 

In [ ]:
#######
# !!!! YOUR CODE HERE !!!!

#######



---



### **(b) Train the model with different strategies:** (3 points)
###  **#1.** Train three models, one without adversarial training, one with adversarial training using `fgsm`, and last one with adversarial training using `pgd`. 

* Use attack budget `epsilon` of 0.05.

* Use a mini-batch of size 512, and train for 20 epochs with learning rate $10^{-2}$, and early stopping tolerance of $10^{-6}$. Report the loss in each case.

* For `pgd`, use the value of step-size `alpha=0.01` and number of iterations `num_iter=100` when training.

In [ ]:
#######
# !!!! YOUR CODE HERE !!!!

#######



---



### **#2.**Measuring performance: In this part we will be evaluate the trained models for different test scenarios. 


### Print the accuracy of each of the three trained models on the clean test dataset. 

In [ ]:
#######
# !!!! YOUR CODE HERE !!!!

#######

### **#3.** Using the same test dataset, perform adversarial attack to compute robust accuracy for each of the three models. Report the robust accuracy of each model for both:
###(a) FGSM attack
###(b) PGD attack

### To create PGD attack examples, use `alpha=0.01`, `num_iter=100`.

In [ ]:
#######
# !!!! YOUR CODE HERE !!!!

#######



---



---



### **(c) FastLin Bound Propagation Algorithm:** In this part, we will use Fast Linear (Fast-Lin) algorithm to get lower and upper bounds for each layer of the neural network. (2.5 points)

**Note:** See Section 3.3 and algorithm 1 in the appendix (Section D: Algorithms) from this paper: \
[Weng etal, Towards Fast Computation of Certified Robustness for ReLU Networks, ICML 2018](https://arxiv.org/pdf/1804.09699.pdf)


### Define a function `fast_linear_bound` that takes as input an ordered list of neural network model layers (`model_layers`), a single input example (`x`), attack budget (`epsilon`). Consider the $l_\infty$ norm ball for this activity. Return a list of tuples of `pre-activation` lower and upper bound tensors for each layer. 
### Also keep in mind to set device when declaring new Tensors.

In [ ]:
#######
# !!! YOUR CODE HERE !!!

#######

In [ ]:
# !!DO NOT EDIT!!
sample_epsilon = 0.2
# unit test - 1
x_1 = torch.tensor([[0.1, 0.9]], device=device)
test_bounds_1 = fast_linear_bound(test_model_layers, x_1, sample_epsilon)
assert torch.all(torch.eq(torch.round(test_bounds_1[0][0], decimals=2), torch.tensor([[0.0000, 0.7000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[0][1], decimals=2), torch.tensor([[0.3000, 1.0000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[1][0], decimals=2), torch.tensor([[0.0000, 1.4000, 1.2000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[1][1], decimals=2), torch.tensor([[0.4500, 2.6000, 1.5000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[2][0], decimals=2), torch.tensor([[3.100, -0.5000,  2.4000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[2][1], decimals=2), torch.tensor([[6.25000, -0.2000, 4.0500]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[3][0], decimals=2), torch.tensor([[4.8000, 4.3000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_1[3][1], decimals=2), torch.tensor([[8.7700, 8.9500]], device=device)))

# unit test - 2
x_2 = torch.tensor([[0.4, 0.5]], device=device)
test_bounds_2 = fast_linear_bound(test_model_layers, x_2, sample_epsilon)
assert torch.all(torch.eq(torch.round(test_bounds_2[0][0], decimals=2), torch.tensor([[0.2000, 0.3000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[0][1], decimals=2), torch.tensor([[0.6000, 0.7000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[1][0], decimals=2), torch.tensor([[0.4000, 1.0000, 0.8000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[1][1], decimals=2), torch.tensor([[1.0000, 2.6000, 1.2000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[2][0], decimals=2), torch.tensor([[0.4000, -0.3000, 0.8000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[2][1], decimals=2), torch.tensor([[4.6000, 0.1000, 3.0000]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[3][0], decimals=2), torch.tensor([[1.2300, 0.4300]], device=device)))
assert torch.all(torch.eq(torch.round(test_bounds_2[3][1], decimals=2), torch.tensor([[6.7000, 6.8000]], device=device)))



---



### **(d) Comparison of certification strategies** (2.5 points)
### **#1.** Get the functions  `bound_propagation` and `binary_search` that you defined in HW3-Q3(d). Refactor the function `binary_search` so that it takes one additional input parameter: `bound_function` which represents a function that can be used to get the bounds, and modify your implementation of `binary_search` so that it uses the parameter `bound_function` for getting the bounds.

In [ ]:
#######
# !!! YOUR CODE HERE !!!

#######

### **#2.** Now, consider the first example from `example_data_flattened`. For this example find the value of certified epsilon (using `binary_search`) using (1) IBP (2) Fast-Lin on the standard model that you trained in part (b). Compare your results.

In [ ]:
#######
# !!! YOUR CODE HERE !!!

#######

### **#3.** Repeat the same activity as above but use the robust model that you trained on PGD adversarial example. Compare you results.

In [ ]:
#######
# !!! YOUR CODE HERE !!!

#######

---
---